In [7]:
import pysam
import os
import re

In [8]:
class paired_read:
    def __init__(self, read):
        self.read_name=read.query_name
        self.reads=list()
        self.reads.append(read)
        
    def add_read(self, read):
        if self.read_name==read.query_name:
            self.reads.append(read)
    
    def is_junctsite_paired_read(self):
        return len(self.reads)==3

In [26]:
class DNAEnd:
    def __init__(self, chr, pos:int, direct):
        self.chr=chr
        self.pos=pos
        self.direct = direct
    
    def get_id(self):
        return(f"{self.chr}_{self.pos}_{self.direct}")
    
    def is_equal(self, new_end):
        return self.chr==new_end.chr and self.pos==new_end.pos and self.direct==new_end.direct

# The junction site only defined the 
class JunctionSite:
    def __init__(self, end1:DNAEnd, end2:DNAEnd):
        self.end1 = end1
        self.end2 = end2

    def get_id(self):
        a = [self.end1.get_id(), self.end2.get_id()]
        a.sort()
        return ("#".join(a))


        

        

In [14]:
# example junction site
end1 = DNAEnd("chr12",93455748,"+")
end2 = DNAEnd("chr12",93455752,"-")
js1= JunctionSite(end1, end2)
print(js1.get_id())

chr12_93455748_+#chr12_93455752_-


In [15]:
# Analyis the read to return key information about junction site
# s_len: the length of the mismatched region
# m_len: the length of match region
# end_type: sm/ms
# DNEEnd object based on read type
def get_end_infor_by_alignment(read):
    m=re.match(r"^(\d+)[S](\d+)M$",read.cigarstring)
    n=re.match(r"^(\d+)M(\d+)[S]$",read.cigarstring)
    if m:
        s_len = int(m.groups(0)[0])
        m_len = int(m.groups(0)[1])
        end_type = "sm"
        return ([s_len,m_len,end_type,DNAEnd(read.reference_name,read.reference_start,"+")])
    elif n:
        s_len = int(n.groups(0)[1])
        m_len = int(n.groups(0)[0])
        end_type = "ms"
        return ([s_len,m_len,end_type,DNAEnd(read.reference_name,read.reference_start+int(n.groups(0)[0]),"-")])
    else:
        return ([None,None,None,None])

# In general, there are three alignments for one paired read 
# In which two alignments from one read can be used to predect the two ends
# these two alignments usually have the same next_reference_start
# The third one is the alignment of the other read
# As the alignment in sam file already on the + strand, 
# it doesn't matter wheter joinsite is in read1/read2 or is reverse or not
def create_junction_site_by_read_list(read_ls):
    if (len(read_ls)!=3): ([None,None])
    # get the nojs_read location
    next_location_ls=[]
    nojs_read_loc=-1
    for read in read_ls:
        if read.next_reference_start in next_location_ls:
            nojs_read_loc = read.next_reference_start
            break
        else:
            next_location_ls.append(read.next_reference_start)
    if nojs_read_loc==-1:
        return ([None,None,None])
    # split the js_read and nojs_read
    js_read=[]  #usually containing two alignments
    nojs_read=[] #only containing one alignment
    for read in read_ls:
        if read.reference_start==nojs_read_loc:
            nojs_read.append(read)
        else:
            js_read.append(read)
    if len(js_read)!=2:
        return ([None,None,None])
    else:
        # sort list by reference start loc to fix the order of
        js_read = sorted(js_read, key=lambda x: x.reference_start)             
    # predict the end type
    [s_len_1,m_len_1,end_type_1,end1]=get_end_infor_by_alignment(js_read[0])
    [s_len_2,m_len_2,end_type_2,end2]=get_end_infor_by_alignment(js_read[1])
    is_read1 = js_read[0].is_read1
    if (end1 and end2):
        # In many cases the junction site may have some bases so the alignments may overlap in these regions
        # junctionsite read ---ATXXXXGC--- 
        # alignment to two genome locus: ---ATXXXX---, locs: ---XXXXGC--- , 
        # The sigar1:---6M2S---, sigar2: ---2S6M--- , 
        # The overlap_base_num here is 4
        # Only modify the first end object to correct the overlap
        overlap_bases_num = m_len_2-s_len_1
        if end_type_1=="sm":
            end1.pos+=overlap_bases_num
        elif end_type_1=="ms":
            end1.pos-=overlap_bases_num
        return ([is_read1, JunctionSite(end1,end2),overlap_bases_num])
    else:
        return ([None,None,None])

# [is_read1, junction_site] = create_junction_site_by_read_list(cur_paired_read_ls[0].reads)
# print(junction_site.get_id())

In [16]:
def sam_to_junction_site_dic(bam, max_alignment_num=100000):
    BAM = pysam.AlignmentFile(bam, 'rb')
    cur_read_name=""
    junction_site_dic = {}
    cur_paired_read = None
    alignment_idx=0
    for read in BAM:
        alignment_idx+=1
        if max_alignment_num<=0 or alignment_idx<max_alignment_num:
            if cur_read_name=="":
                cur_paired_read=paired_read(read)
            elif cur_read_name != read.query_name:
                # new paired reads
                if cur_paired_read.is_junctsite_paired_read():
                    [is_read1, junction_site,overlap_bases_num] = create_junction_site_by_read_list(cur_paired_read.reads)
                    if junction_site:
                        js_site_id = junction_site.get_id()
                        if js_site_id not in junction_site_dic.keys():
                            junction_site_dic[js_site_id] = {"junction_site": junction_site, "read1": 0, "read2": 0, "overlap_bp":overlap_bases_num}
                        if is_read1:
                            junction_site_dic[js_site_id]["read1"]+=1
                        else:
                            junction_site_dic[js_site_id]["read2"]+=1
                cur_paired_read=paired_read(read)
            else:
                cur_paired_read.add_read(read)
            cur_read_name = read.query_name
        else:
            break
    # For the last paired_read
    if cur_paired_read and cur_paired_read.is_junctsite_paired_read():
        [is_read1, junction_site, overlap_bases_num] = create_junction_site_by_read_list(cur_paired_read.reads)
        if junction_site:
            js_site_id = junction_site.get_id()
            if js_site_id not in junction_site_dic.keys():
                junction_site_dic[js_site_id] = {"junction_site": junction_site, "read1": 0, "read2": 0, "overlap_bp":overlap_bases_num}
            if is_read1:
                junction_site_dic[js_site_id]["read1"]+=1
            else:
                junction_site_dic[js_site_id]["read2"]+=1
    return (junction_site_dic) 


def get_eccDNA_type_by_junctionsite(junction_site, max_single_eccDNA_length=1000000):
    if junction_site.end1.chr==junction_site.end2.chr:
        if (abs(junction_site.end1.pos-junction_site.end1.pos)<max_single_eccDNA_length):
            if (junction_site.end1.pos<junction_site.end2.pos and junction_site.end1.direct=="+" and junction_site.end2.direct=="-"):
                return("single_eccDNA")
            if (junction_site.end2.pos<junction_site.end1.pos and junction_site.end2.direct=="+" and junction_site.end1.direct=="-"):
                return("single_eccDNA")
            return("in_chr_unknown")
        else:
            return("in_chr_unknown")
    else:
        return("across_chr")
    
def get_eccDNA_region_by_junctionsite(junction_site):
    if get_eccDNA_type_by_junctionsite(junction_site)=="single_eccDNA":
        ecc_length=abs(junction_site.end1.pos-junction_site.end2.pos)+1
        if (junction_site.end1.pos<junction_site.end2.pos and junction_site.end1.direct=="+" and junction_site.end2.direct=="-"):
            return(f"{junction_site.end1.chr},{junction_site.end1.pos},{junction_site.end2.pos}")
        if (junction_site.end2.pos<junction_site.end1.pos and junction_site.end2.direct=="+" and junction_site.end1.direct=="-"):
            return(f"{junction_site.end1.chr},{junction_site.end2.pos},{junction_site.end1.pos}")
    else:
        return "-"

# # bam = "/home/hqyone/mnt/2tb/eccDNA/data-raw/test.bam"
# bam = "/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D21010704/bam/D21010704_hg38_sort_du_byname.bam"

# junction_site_dic = sam_to_junction_site_dic(bam, max_alignment_num=0)

# #print junction_site_dic to file
# output = "/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D21010704/bam/D21010704_junction_site.csv"
# eccBed = "/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D21010704/bam/D21010704_ecc.bed"
# with open(output,'w') as OUT, open(eccBed,'w') as BED:
#     OUT.write(f"id\tread1\tread2\tecc_type\tecc_region\toverlap_bases_num\n")
#     for js_key in junction_site_dic.keys():
#         js_infor = junction_site_dic[js_key]
#         js_obj = js_infor["junction_site"]
#         read1 = js_infor["read1"]
#         read2 = js_infor["read2"]
#         ecc_type = get_eccDNA_type_by_junctionsite(js_obj)
#         ecc_region = get_eccDNA_region_by_junctionsite(js_obj)
#         overlap_bases_num = js_infor["overlap_bp"]
#         if (read1+read2>500):
#             OUT.write(f"{js_key}\t{read1}\t{read2}\t{ecc_type}\t{ecc_region}\t{overlap_bases_num}\n")
#             if (ecc_type =="single_eccDNA"):
#                 BED.write(ecc_region.replace(",","\t")+"\t"+js_key+"\t"+str(read1+read2)+"\t+\n")
        
        

In [19]:
def process_bam_file(bam, outcsv, eccBed, min_support_read_num=300):
    junction_site_dic = sam_to_junction_site_dic(bam, max_alignment_num=0)
    with open(outcsv,'w') as OUT, open(eccBed,'w') as BED:
        OUT.write(f"id\tread1\tread2\tecc_type\tecc_region\toverlap_bases_num\n")
        for js_key in junction_site_dic.keys():
            js_infor = junction_site_dic[js_key]
            js_obj = js_infor["junction_site"]
            read1 = js_infor["read1"]
            read2 = js_infor["read2"]
            ecc_type = get_eccDNA_type_by_junctionsite(js_obj)
            ecc_region = get_eccDNA_region_by_junctionsite(js_obj)
            overlap_bases_num = js_infor["overlap_bp"]
            if (read1+read2>min_support_read_num):
                OUT.write(f"{js_key}\t{read1}\t{read2}\t{ecc_type}\t{ecc_region}\t{overlap_bases_num}\n")
                if (ecc_type =="single_eccDNA"):
                    BED.write(ecc_region.replace(",","\t")+"\t"+js_key+"\t"+str(read1+read2)+"\t+\n")


def process_bam_files(bam_ls, output_dir):
    for bam in bam_ls:
        id =bam.split("/")[-3]
        print(id)
        output_csv = f"{output_dir}/{id}/{id}_junction_site.csv"
        output_bed = f"{output_dir}/{id}/{id}_junction_ecc.bed"
        process_bam_file(bam, output_csv, output_bed,min_support_read_num=100)




In [20]:
junction_site_lsimport glob
output_dir="/home/hqyone/mnt/2tb/eccDNA/results/total"
path="/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna"  #input from the user 
bam_ls = glob.glob(f"{path}/**/*_sort_du_byname.bam", recursive=True)
process_bam_files(bam_ls,output_dir)

D21010704


[E::hts_idx_load3] Could not load local index file '/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D21010704/bam/D21010704_hg38_sort_du_byname.bam.bai' : No such file or directory


D21010705


[E::hts_idx_load3] Could not load local index file '/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D21010705/bam/D21010705_hg38_sort_du_byname.bam.bai' : No such file or directory


D21010706


...

D22040610


[E::idx_find_and_load] Could not retrieve index file for '/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D22040610/bam/D22040610_hg38_sort_du_byname.bam'


D22040611


[E::idx_find_and_load] Could not retrieve index file for '/home/hqyone/mnt/2tb/eccDNA/data-final/eccdna/D22040611/bam/D22040611_hg38_sort_du_byname.bam'


test_sample_id


In [32]:


def create_end_by_id(id):
    [chr, pos, direct]=id.split("_")
    return(DNAEnd(chr, int(pos),direct))

def isMatch(end1, end2, min_dist=10, max_distance=50000):
    if end1.chr==end2.chr and end1.direct!=end2.direct:
        dist = abs(end1.pos-end2.pos+1)
        if (dist>min_dist and dist<max_distance):
            return True
    return False

def search_matched_junction_sites(start_DNAEnd, end_DNAEnd, junction_site_ls,out_list):
    out_list.append(end_DNAEnd)
    for site_id in junction_site_ls:
        end1=create_end_by_id(site_id.split("#")[0])
        end2=create_end_by_id(site_id.split("#")[1])
        if (isMatch(end_DNAEnd, end1)):
            out_list.append(end1)
            if (not end1.is_equal(start_DNAEnd)):
                search_matched_junction_sites(start_DNAEnd, end2,  junction_site_ls, out_list)
        elif(isMatch(end_DNAEnd, end2)):
            out_list.append(end2)
            if (not end2.is_equal(start_DNAEnd)):
                search_matched_junction_sites(start_DNAEnd, end1,  junction_site_ls, out_list)
        
start_js_site_id = "chr12_133048519_+#chr9_126376421_+"
junction_site_ls = ["chr12_133048519_+#chr9_126376421_+", "chr12_133048739_-#chr9_126376860_-"]

start_dna_end= create_end_by_id(start_js_site_id.split("#")[0])
next_dna_end= create_end_by_id(start_js_site_id.split("#")[1])
multple_region_eccDNA_ls=[start_dna_end]

search_matched_junction_sites(start_dna_end, next_dna_end, junction_site_ls, multple_region_eccDNA_ls)
for end in multple_region_eccDNA_ls:
    print(end.get_id())

# def search_multple_region_eccDNA(start_junction_site_id, junction_site_ls, multple_region_eccDNA_ls):
#     start_dna_end= create_end_by_id(start_junction_site_id.split("#")[0])
#     next_dna_end= create_end_by_id(start_junction_site_id.split("#")[1])
#     for site_id in junction_site_ls:
#         end1=create_end_by_id(site_id.split("#")[0])
#         end2=create_end_by_id(site_id.split("#")[1])
#         if (isMatch(next_dna_end, end1)){
#             new_js_id = f"{end1.get_id()}#{end2.get_id()}"
#             multple_region_eccDNA_ls.append(multple_region_eccDNA_ls.append(search_multple_region_eccDNA(),junction_site_ls))
#         }elif(isMatch(next_dna_end, end2))
#             new_js_id = f"{end2.get_id()}#{end1.get_id()}"
#             multple_region_eccDNA_ls.append(multple_region_eccDNA_ls.append(search_multple_region_eccDNA(),junction_site_ls))


# def search_downstream_junction_site()

chr12_133048519_+
chr9_126376421_+
chr9_126376860_-
chr12_133048739_-
chr12_133048519_+
